In [1]:
%%capture
!pip install SimpleITK
# !pip install skan
!pip install git+https://github.com/jni/skan
!pip install tifffile

In [6]:
import SimpleITK as sitk
from skimage.morphology import skeletonize, thin, medial_axis#, skeletonize_3d
from scipy import ndimage
from skan import csr, Skeleton
from skan import summarize
from skan import draw
import tifffile as tiff
import asyncio

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import cv2
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import glob
from tqdm.notebook import tnrange
import shutil
from google.colab import files
import os

In [7]:
def plot_3d_nodes_and_branches_mpl(df_nodes, df_segments):
    """Plots the 3D nodes and branches using Matplotlib."""
    fig = plt.figure(figsize=(10, 10))
    ax = fig.add_subplot(111, projection='3d')

    # Plot nodes
    ax.scatter(df_nodes['node_coordinate_x'], df_nodes['node_coordinate_y'], df_nodes['node_coordinate_z'], c='red', marker='o', label='Nodes')

    # Plot branches
    for _, row in df_segments.iterrows():
        path = np.array(row['path-coords'])
        ax.plot(path[:, 2], path[:, 1], path[:, 0], color='blue', linewidth=1)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.legend()
    plt.show()



def plot_3d_nodes_and_branches_plotly(df_nodes, df_segments):
    """
    Plots 3D nodes and branches using Plotly.

    Args:
        df_nodes (pd.DataFrame): DataFrame containing node data.
        df_segments (pd.DataFrame): DataFrame containing segments data.
    """
    fig = go.Figure()

    # Plot nodes
    fig.add_trace(go.Scatter3d(
        x=df_nodes['node_coordinate_x'],
        y=df_nodes['node_coordinate_y'],
        z=df_nodes['node_coordinate_z'],
        mode='markers',
        marker=dict(size=5, color='red'),
        name='Nodes'
    ))

    # Plot branches
    for _, row in df_segments.iterrows():
        path = np.array(row['path-coords'])
        if path.shape[1] == 3:  # Ensure correct shape
            fig.add_trace(go.Scatter3d(
                x=path[:, 2], y=path[:, 1], z=path[:, 0],  # Swap coordinates to match nodes
                mode='lines',
                line=dict(color='blue', width=2),
                name='Branches'
            ))

    fig.update_layout(
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        title='3D Nodes and Branches Visualization'
    )
    fig.show()


def tiffs_to_3d_numpy(folderpath):
    """
    Converts tiff files to a 3D numpy array.

    Args:
        folderpath (str): path to folder containing tiff files.

    Returns:
        np.array: 3D numpy array containing image data.
    """
    filelist = [os.path.join(folderpath, f) for f in os.listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f))]

    # Sort files by numerical order based on filename
    filelist.sort(key=lambda x: int(x[-8:-5]))  # Sort by the 3-digit number in filenames

    sorted_filelist = [f for f in filelist]

    # Read the TIFF images into a numpy array
    return np.array([plt.imread(fname) for fname in sorted_filelist])


def compute_porosity(img):
    """
    Computes porosity as the ratio of black pixels to total pixels.

    Args:
        img (np.array): binary image (0 for black, 1 for white).

    Returns:
        float: Porosity value.
    """
    num_white_pix = np.sum(img == 1)
    num_black_pix = np.sum(img == 0)
    return num_black_pix / (num_white_pix + num_black_pix)


def compute_distance_transform(segmentation_data):
    """
    Computes the 3D distance transform of the segmentation data.

    Args:
        segmentation_data (np.array): 3D binary segmentation data.

    Returns:
        np.array: 3D distance transform of the binary image.
    """
    return ndimage.distance_transform_edt(segmentation_data)


def skeletonize_data(segmentation_data, dist_trans_3d):
    """
    Skeletonizes the binary image and summarizes the skeleton using Skan.

    Args:
        segmentation_data (np.array): Binary image data.
        dist_trans_3d (np.array): 3D distance transform of the binary image.

    Returns:
        pd.DataFrame: DataFrame containing skeleton data.
    """
    skimage_skeleton = skeletonize(segmentation_data, method='lee')
    skan_skeleton = Skeleton(skeleton_image=skimage_skeleton * dist_trans_3d)
    df_skeleton = summarize(skan_skeleton)

    # Rename and scale columns
    df_skeleton.rename(columns={
        'image-coord-src-0': 'src-x',
        'image-coord-src-1': 'src-y',
        'image-coord-src-2': 'src-z',
        'image-coord-dst-0': 'dst-x',
        'image-coord-dst-1': 'dst-y',
        'image-coord-dst-2': 'dst-z',
        'mean-pixel-value': 'thickness'
    }, inplace=True)

    # Scale thickness and distances
    df_skeleton[['branch-distance', 'euclidean-distance']] *= (1.485)
    df_skeleton[['thickness']] *= (2 * 1.485)

    # Drop unnecessary columns
    df_skeleton.drop(columns=['stdev-pixel-value', 'coord-src-0', 'coord-src-1', 'coord-src-2', 'coord-dst-0', 'coord-dst-2', 'coord-dst-1'], inplace=True)

    return df_skeleton


def compute_branch_coordinates(df_skeleton, skan_skeleton):
    """
    Computes the 3D path coordinates for branches in the skeleton.

    Args:
        df_skeleton (pd.DataFrame): DataFrame containing skeleton data.
        skan_skeleton (Skeleton): Skan skeleton object.

    Returns:
        pd.DataFrame: Updated DataFrame with path coordinates.
    """
    branch_path_coords = [np.array(skan_skeleton.path_coordinates(i))[:, [2, 1, 0]].tolist() for i in range(len(df_skeleton))]
    df_skeleton['path-coords'] = branch_path_coords

    return df_skeleton


def compute_segments(df_skeleton):
    """
    Computes segment data based on skeleton and path coordinates.

    Args:
        df_skeleton (pd.DataFrame): DataFrame containing skeleton data.

    Returns:
        pd.DataFrame: DataFrame containing segments data.
    """
    segments = df_skeleton[['thickness', 'branch-distance', 'euclidean-distance', 'path-coords']].copy()
    segments['source_node_id'] = df_skeleton['node-id-src']
    segments['destination_node_id'] = df_skeleton['node-id-dst']
    segments['tortuosity'] = segments['branch-distance'] / segments['euclidean-distance']
    segments.loc[segments['path-coords'].apply(lambda x: x[0] == x[-1]), 'tortuosity'] = -1

    return segments


def compute_node_coordinates(df_skeleton):
    """
    Computes final node coordinates from skeleton data.

    Args:
        df_skeleton (pd.DataFrame): DataFrame containing skeleton data.

    Returns:
        pd.DataFrame: DataFrame containing unique node coordinates.
    """
    final_nodes = pd.concat([
        df_skeleton[['node-id-src', 'src-x', 'src-y', 'src-z']].rename(columns={'node-id-src': 'node_id', 'src-x': 'node_coordinate_x', 'src-y': 'node_coordinate_y', 'src-z': 'node_coordinate_z'}),
        df_skeleton[['node-id-dst', 'dst-x', 'dst-y', 'dst-z']].rename(columns={'node-id-dst': 'node_id', 'dst-x': 'node_coordinate_x', 'dst-y': 'node_coordinate_y', 'dst-z': 'node_coordinate_z'})
    ], ignore_index=True).drop_duplicates().reset_index(drop=True)

    return final_nodes

In [ ]:
import os
import pandas as pd
from zipfile import ZipFile

stereom_ids = [1, 2, 3, 4, 5, 6]  # Optionally run on multiple STEREOMs
rve_ids = [1, 2, 3, 4]

porosity_dict = {}

for stereom_id in stereom_ids:
    for rve_id in rve_ids:
        zip_path = f"./STEREOM {stereom_id} RVE {rve_id}.zip"
        assert os.path.exists(zip_path), f"Zip file not found: {zip_path}"

        os.system(f'unzip "{zip_path}"')

        folder_id_lower = f"/content/STEREOM {stereom_id} RVE {rve_id}/Threshold/"
        folder_id_upper = f"/content/STEREOM {stereom_id} RVE {rve_id}/THRESHOLD/"

        folder_id = folder_id_lower if os.path.exists(folder_id_lower) else folder_id_upper

        # Unzip the dataset

        # raise AssertionError(zip_path)
        # with ZipFile(zip_path, 'r') as zip_ref:
        #     zip_ref.extractall("/content/")

        # os.system(f'unzip "{zip_path}"')


        # Process TIFF images
        segmentation_data_np = tiffs_to_3d_numpy(folder_id)
        porosity = compute_porosity(segmentation_data_np)
        porosity_dict[f"STEREOM_{stereom_id}_RVE_{rve_id}"] = porosity * 100
        print(f"STEREOM {stereom_id}, RVE {rve_id} - Porosity = {porosity * 100:.2f}%")

        # Compute distance transform
        dist_trans_3d = compute_distance_transform(segmentation_data_np)

        # Skeletonize data
        df_skeleton = skeletonize_data(segmentation_data_np, dist_trans_3d)

        # Compute branch coordinates
        df_skeleton = compute_branch_coordinates(df_skeleton, Skeleton(skeleton_image=skeletonize(segmentation_data_np, method='lee') * dist_trans_3d))

        # Compute segments
        segments = compute_segments(df_skeleton)

        # Save segments to CSV
        segments_filename = f"STEREOM_{stereom_id}_RVE_{rve_id}_segments.csv"
        segments.to_csv(segments_filename, index=False)

        # Compute node coordinates
        final_nodes_3d = compute_node_coordinates(df_skeleton)

        # Save nodes to CSV
        nodes_filename = f"STEREOM_{stereom_id}_RVE_{rve_id}_nodes.csv"
        final_nodes_3d.to_csv(nodes_filename, index=False)

        # Plot nodes and branches
        plot_3d_nodes_and_branches_mpl(final_nodes_3d, segments)

        avg_thickness = df_skeleton['thickness'].mean()
        avg_branch_distance = df_skeleton['branch-distance'].mean()
        avg_euclidean_distance = df_skeleton['euclidean-distance'].mean()

        print(f"STEREOM {stereom_id}, RVE {rve_id} - Average Thickness: {avg_thickness:.3f}")
        print(f"STEREOM {stereom_id}, RVE {rve_id} - Average Branch Distance: {avg_branch_distance:.3f}")
        print(f"STEREOM {stereom_id}, RVE {rve_id} - Average Euclidean Distance: {avg_euclidean_distance:.3f}")

        # Download the generated CSV files
        # files.download(segments_filename)
        # files.download(nodes_filename)

# Save porosity values to a DataFrame
porosity_df = pd.DataFrame(list(porosity_dict.items()), columns=["Dataset", "Porosity (%)"])
print(porosity_df)
porosity_df.to_csv("porosity_values.csv", index=False)
files.download("porosity_values.csv")


In [11]:
import os
from zipfile import ZipFile

zip_filename = "/content/all_results.zip"

with ZipFile(zip_filename, 'w') as zipf:
    for stereom_id in range(1, 7):
        for rve_id in range(1, 5):
            nodes_filename = f"/content/STEREOM_{stereom_id}_RVE_{rve_id}_nodes.csv"
            segments_filename = f"/content/STEREOM_{stereom_id}_RVE_{rve_id}_segments.csv"

            if os.path.exists(nodes_filename):
                zipf.write(nodes_filename, os.path.basename(nodes_filename))
            if os.path.exists(segments_filename):
                zipf.write(segments_filename, os.path.basename(segments_filename))

print(f"Zipped all nodes and segments CSV files into {zip_filename}")


Zipped all nodes and segments CSV files into /content/all_results.zip


In [ ]:
# might take some time
# plot_3d_nodes_and_branches_plotly(final_nodes_3d, segments)

In [ ]:
# # /content/Thresholds/Threshold model 1/ nodes.csv
# files.download(folder_id+"_nodes.csv")
# files.download(folder_id+"_segments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>